# Framework for predictions and portfolio forming

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time

# import the parquet library
import pyarrow.parquet as pq

# import model libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
# load 'basemodel.parquet'
#df = pd.read_parquet('basemodel.parquet')
df = pd.read_parquet('/kaggle/input/sign-prediction-datasets/basicmarket.parquet')
prediction_cols = []
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,STR,LTURNOVER,IMOM,MOM,LTR,PCTHIGH,IVOL2,IVOL3,MVOL,MVOL2,MVOL3,LMKT2,LMKT3,MMOM,MIMOM,MLTR,LSPREAD
0,10000,1987-03-31,-0.384615,973.25000,0.0,0.0,0.0,1.0,0,0,0.0482,0.000612,0,0.000000,0.100694,-1.089044,-2.079441,-0.297252,0.091549,0.000830,0.002413,0.545027,0.771919,0.434190,0.1289,-0.0278,0.186212,0.112764,0.459110,0.076923
1,10000,1987-04-30,-0.062500,912.44134,0.0,0.0,0.0,0.0,0,1,0.0211,0.003465,0,-0.384615,0.285384,-1.459321,-2.390877,0.014185,0.062500,0.000612,0.000830,0.672597,0.545027,0.771919,0.0482,0.1289,0.179935,-0.025601,0.498953,0.625000
2,10000,1987-05-31,-0.066667,851.59375,0.0,0.0,0.0,0.0,0,2,-0.0167,0.001893,0,-0.062500,0.256358,-1.633155,-2.772587,-0.089613,0.075378,0.003465,0.000612,1.565461,0.672597,0.545027,0.0211,0.0482,0.208747,0.032263,0.488026,0.133330
3,10001,1987-03-31,0.036800,6317.62500,0.0,0.0,0.0,1.0,0,0,0.0482,0.001627,1,-0.074074,0.368315,0.071156,0.140122,0.020203,0.892857,0.001556,0.001527,0.545027,0.771919,0.434190,0.1289,-0.0278,0.186212,0.112764,0.459110,0.120000
4,10001,1987-04-30,-0.039216,6069.87500,0.0,0.0,0.0,0.0,0,0,0.0211,0.001597,0,0.036800,0.217962,0.043187,0.038273,0.045090,0.910714,0.001627,0.001556,0.672597,0.545027,0.771919,0.0482,0.1289,0.179935,-0.025601,0.498953,0.078431


In [3]:
# select the columns to be used for prediction
X_col = ['bull_D', 'bear_D', 'bull_W', 'bear_W', 'bull_M', 'bear_M', 'LMKT', 'IVOL', # initial columns
         'STR',	'LTURNOVER', 'IMOM', 'MOM',	'LTR', 'PCTHIGH', 'IVOL2', 'IVOL3', 'LSPREAD',	# stock specific columns
         'MVOL', 'MVOL2', 'MVOL3', 'LMKT2',	'LMKT3', 'MMOM', 'MIMOM', 'MLTR'] # market specific columns

In [4]:
# scale the data for faster coeficient convergence
scaler = MinMaxScaler()

df[X_col] = scaler.fit_transform(df[X_col])

In [5]:
# Convert 'date' to datetime format (if not already done) and sort the DataFrame
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a 'year' column based on the 'date' column
df['year'] = df['date'].dt.year

# Paper Replication - OLS and Logit, Expanding Window - No Hyperparameters
- They start with out of sample forecasting in 1932
- models will be named model_default

### Linear Regression (Pooled OLS)

In [6]:
#################################
# OLS, default, exp window
#################################

model_name = 'ols_default'


# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

for year in range(df['year'].min() + 6, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Define the training data up until this year
    train_data = df[df['year'] < year]
    
    X_train = train_data[X_col]
    y_train = train_data['y']
    
    # Train the Linear Regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Time: {iteration_time:.2f} seconds")

Year 1933 - Time: 0.13 seconds
Year 1934 - Time: 0.18 seconds
Year 1935 - Time: 0.22 seconds
Year 1936 - Time: 0.20 seconds
Year 1937 - Time: 0.19 seconds
Year 1938 - Time: 0.23 seconds
Year 1939 - Time: 0.26 seconds
Year 1940 - Time: 0.28 seconds
Year 1941 - Time: 0.26 seconds
Year 1942 - Time: 0.29 seconds
Year 1943 - Time: 0.29 seconds
Year 1944 - Time: 0.31 seconds
Year 1945 - Time: 0.34 seconds
Year 1946 - Time: 0.37 seconds
Year 1947 - Time: 0.44 seconds
Year 1948 - Time: 0.46 seconds
Year 1949 - Time: 0.43 seconds
Year 1950 - Time: 0.43 seconds
Year 1951 - Time: 0.45 seconds
Year 1952 - Time: 0.45 seconds
Year 1953 - Time: 0.48 seconds
Year 1954 - Time: 0.49 seconds
Year 1955 - Time: 0.53 seconds
Year 1956 - Time: 0.54 seconds
Year 1957 - Time: 0.55 seconds
Year 1958 - Time: 0.58 seconds
Year 1959 - Time: 0.58 seconds
Year 1960 - Time: 0.61 seconds
Year 1961 - Time: 0.62 seconds
Year 1962 - Time: 0.66 seconds
Year 1963 - Time: 0.67 seconds
Year 1964 - Time: 0.69 seconds
Year 196

# My Experiments

## Machine Learning - Hyperparameter Tuning included in the process
- models to be named 'model_clas/reg_exp/roll'

### First expanding, then rolling
start predicting for 1932, expand the window until you reach X years, then roll it

##### 5 year window

In [7]:
# set the length of the rolling window
rolling_window = 5 # years

In [8]:
#################################
# RIDGE REGRESSION
#################################


model_name = 'ridge_reg_roll5'  # Name for storing Ridge regression predictions
start_time2 = time.time()  # Start timing


# Predefined set of alpha values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100]  # Alpha for Ridge

# Update the column name for storing Ridge regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 6


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Prepare training and tuning datasets
    train_start_year = max(year - rolling_window, df['year'].min())
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_accuracy = 0  # Initialize with 0 for accuracy comparison
    
    # Hyperparameter tuning with accuracy on top and bottom deciles
    for hp1 in HP1:
        model = Ridge(alpha=hp1)
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)
        
        # Identify top and bottom deciles
        decile_thresholds = np.percentile(predictions, [10, 90])
        top_bottom_decile_mask = (predictions <= decile_thresholds[0]) | (predictions >= decile_thresholds[1])
        
        # Filter tuning dataset based on deciles
        filtered_y_tune = y_tune[top_bottom_decile_mask]
        filtered_predictions = predictions[top_bottom_decile_mask]
        
        # Assuming binary classification, convert continuous predictions to binary
        # This conversion logic might need adjustment based on your specific use case
        binary_predictions = (filtered_predictions >= 0.5).astype(int)
        
        # Calculate accuracy for filtered predictions
        decile_accuracy = accuracy_score(filtered_y_tune, binary_predictions)
        
        if decile_accuracy > best_accuracy:
            best_accuracy = decile_accuracy
            best_HP1 = hp1
    
    # Retrain with best hyperparameters on the entire training data
    model = Ridge(alpha=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year and update df
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best Alpha: {best_HP1}, Best Decile Accuracy: {round(best_accuracy, 4)}, Time: {iteration_time:.2f} seconds")

end_time2 = time.time()  # End timing
print(f"Total time: {end_time2 - start_time2:.2f} seconds")

Year 1933 - Best Alpha: 100, Best Decile Accuracy: 0.9371, Time: 0.29 seconds
Year 1934 - Best Alpha: 10, Best Decile Accuracy: 0.9975, Time: 0.33 seconds
Year 1935 - Best Alpha: 1, Best Decile Accuracy: 0.9994, Time: 0.33 seconds
Year 1936 - Best Alpha: 0.01, Best Decile Accuracy: 0.9994, Time: 0.32 seconds
Year 1937 - Best Alpha: 0.01, Best Decile Accuracy: 0.9988, Time: 0.31 seconds
Year 1938 - Best Alpha: 100, Best Decile Accuracy: 0.9207, Time: 0.31 seconds
Year 1939 - Best Alpha: 1, Best Decile Accuracy: 0.9909, Time: 0.31 seconds
Year 1940 - Best Alpha: 100, Best Decile Accuracy: 0.9855, Time: 0.31 seconds
Year 1941 - Best Alpha: 0.01, Best Decile Accuracy: 0.9978, Time: 0.31 seconds
Year 1942 - Best Alpha: 1, Best Decile Accuracy: 0.9939, Time: 0.34 seconds
Year 1943 - Best Alpha: 10, Best Decile Accuracy: 0.9978, Time: 0.32 seconds
Year 1944 - Best Alpha: 10, Best Decile Accuracy: 0.9973, Time: 0.32 seconds
Year 1945 - Best Alpha: 1, Best Decile Accuracy: 0.9936, Time: 0.31 se

In [9]:
#################################
# LASSO REGRESSION
#################################


model_name = 'lasso_reg_roll5'  # Name for storing Ridge regression predictions
start_time2 = time.time()  # Start timing


# Predefined set of alpha values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100]  # Alpha for Ridge

# Update the column name for storing Ridge regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 6


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Prepare training and tuning datasets
    train_start_year = max(year - rolling_window, df['year'].min())
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_accuracy = 0  # Initialize with 0 for accuracy comparison
    
    # Hyperparameter tuning with accuracy on top and bottom deciles
    for hp1 in HP1:
        model = Lasso(alpha=hp1)
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)
        
        # Identify top and bottom deciles
        decile_thresholds = np.percentile(predictions, [10, 90])
        top_bottom_decile_mask = (predictions <= decile_thresholds[0]) | (predictions >= decile_thresholds[1])
        
        # Filter tuning dataset based on deciles
        filtered_y_tune = y_tune[top_bottom_decile_mask]
        filtered_predictions = predictions[top_bottom_decile_mask]
        
        # Assuming binary classification, convert continuous predictions to binary
        # This conversion logic might need adjustment based on your specific use case
        binary_predictions = (filtered_predictions >= 0.5).astype(int)
        
        # Calculate accuracy for filtered predictions
        decile_accuracy = accuracy_score(filtered_y_tune, binary_predictions)
        
        if decile_accuracy > best_accuracy:
            best_accuracy = decile_accuracy
            best_HP1 = hp1
    
    # Retrain with best hyperparameters on the entire training data
    model = Lasso(alpha=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year and update df
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best Alpha: {best_HP1}, Best Decile Accuracy: {round(best_accuracy, 4)}, Time: {iteration_time:.2f} seconds")

end_time2 = time.time()  # End timing
print(f"Total time: {end_time2 - start_time2:.2f} seconds")

Year 1933 - Best Alpha: 0.01, Best Decile Accuracy: 0.9996, Time: 0.40 seconds
Year 1934 - Best Alpha: 0.01, Best Decile Accuracy: 0.9969, Time: 0.47 seconds
Year 1935 - Best Alpha: 0.01, Best Decile Accuracy: 0.9994, Time: 0.48 seconds
Year 1936 - Best Alpha: 0.01, Best Decile Accuracy: 0.9994, Time: 0.42 seconds
Year 1937 - Best Alpha: 0.01, Best Decile Accuracy: 1.0, Time: 0.51 seconds
Year 1938 - Best Alpha: 0.01, Best Decile Accuracy: 1.0, Time: 0.43 seconds
Year 1939 - Best Alpha: 0.01, Best Decile Accuracy: 1.0, Time: 0.52 seconds
Year 1940 - Best Alpha: 0.01, Best Decile Accuracy: 0.9995, Time: 0.50 seconds
Year 1941 - Best Alpha: 0.01, Best Decile Accuracy: 0.999, Time: 0.42 seconds
Year 1942 - Best Alpha: 0.01, Best Decile Accuracy: 0.9989, Time: 0.54 seconds
Year 1943 - Best Alpha: 0.01, Best Decile Accuracy: 0.9946, Time: 0.41 seconds
Year 1944 - Best Alpha: 0.01, Best Decile Accuracy: 0.9898, Time: 0.43 seconds
Year 1945 - Best Alpha: 0.01, Best Decile Accuracy: 0.9983, Ti

In [10]:
#################################
# DECISION TREE REGRESSION
#################################

model_name = 'DT_reg_roll5'  # Name for storing Decision Tree regression predictions

# Predefined set of values for hyperparameter tuning (max_depth in this case)
HP1 = [None, 5, 10, 15, 20, 25]  # Possible values for max_depth

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 6


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_accuracy = 0  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = DecisionTreeRegressor(max_depth=hp1)  # Decision Tree Regressor
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)
        
        # Identify top and bottom deciles
        decile_thresholds = np.percentile(predictions, [10, 90])
        top_bottom_decile_mask = (predictions <= decile_thresholds[0]) | (predictions >= decile_thresholds[1])
        
        # Filter tuning dataset based on deciles
        filtered_y_tune = y_tune[top_bottom_decile_mask]
        filtered_predictions = predictions[top_bottom_decile_mask]
        
        # Assuming binary classification, convert continuous predictions to binary
        # This conversion logic might need adjustment based on your specific use case
        binary_predictions = (filtered_predictions >= 0.5).astype(int)
        
        # Calculate accuracy for filtered predictions
        decile_accuracy = accuracy_score(filtered_y_tune, binary_predictions)
        
        if decile_accuracy > best_accuracy:
            best_accuracy = decile_accuracy
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best max_depth value
    model = DecisionTreeRegressor(max_depth=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best max_depth: {best_HP1}, Best ACC: {round(best_accuracy,4)}, Time: {iteration_time:.2f} seconds")


Year 1933 - Best max_depth: 5, Best ACC: 1.0, Time: 1.55 seconds
Year 1934 - Best max_depth: 5, Best ACC: 1.0, Time: 1.61 seconds
Year 1935 - Best max_depth: 5, Best ACC: 1.0, Time: 1.70 seconds
Year 1936 - Best max_depth: 5, Best ACC: 1.0, Time: 1.78 seconds
Year 1937 - Best max_depth: 5, Best ACC: 1.0, Time: 1.67 seconds
Year 1938 - Best max_depth: 5, Best ACC: 1.0, Time: 1.63 seconds
Year 1939 - Best max_depth: 5, Best ACC: 1.0, Time: 1.73 seconds
Year 1940 - Best max_depth: 5, Best ACC: 1.0, Time: 1.76 seconds
Year 1941 - Best max_depth: 5, Best ACC: 1.0, Time: 1.95 seconds
Year 1942 - Best max_depth: 5, Best ACC: 1.0, Time: 1.98 seconds
Year 1943 - Best max_depth: 5, Best ACC: 1.0, Time: 1.96 seconds
Year 1944 - Best max_depth: 5, Best ACC: 1.0, Time: 1.98 seconds
Year 1945 - Best max_depth: 5, Best ACC: 1.0, Time: 1.91 seconds
Year 1946 - Best max_depth: 5, Best ACC: 1.0, Time: 1.97 seconds
Year 1947 - Best max_depth: 5, Best ACC: 1.0, Time: 1.85 seconds
Year 1948 - Best max_dept

In [11]:
#################################
# RANDOM FOREST REGRESSION
#################################
start_time2 = time.time()  # Start timing

model_name = 'RF_reg_roll5'  # Name for storing Random Forest regression predictions

# Predefined set of values for hyperparameter tuning
HP1 = [10, 50, 100, 200]  # Possible values for n_estimators
HP2 = [5, 10, 15, None]  # Possible values for max_depth

# Update the column name for storing Random Forest regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 6


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_HP2 = None
    best_accuracy = -1
    
    # Hyperparameter tuning
    for hp1 in HP1:
        for hp2 in HP2:
            model = RandomForestRegressor(n_estimators=hp1, max_depth=hp2, random_state=42, n_jobs = -1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_tune)  # Predict continuous values
            
            # Identify top and bottom deciles
            decile_thresholds = np.percentile(predictions, [10, 90])
            top_bottom_decile_mask = (predictions <= decile_thresholds[0]) | (predictions >= decile_thresholds[1])

            # Filter tuning dataset based on deciles
            filtered_y_tune = y_tune[top_bottom_decile_mask]
            filtered_predictions = predictions[top_bottom_decile_mask]

            # Assuming binary classification, convert continuous predictions to binary
            # This conversion logic might need adjustment based on your specific use case
            binary_predictions = (filtered_predictions >= 0.5).astype(int)

            # Calculate accuracy for filtered predictions
            decile_accuracy = accuracy_score(filtered_y_tune, binary_predictions)

            if decile_accuracy > best_accuracy:
                best_accuracy = decile_accuracy
                best_HP1 = hp1
                best_HP2 = hp2
    
    # Retrain on the entire training window (excluding tuning year) with the best hyperparameters
    model = RandomForestRegressor(n_estimators=best_HP1, max_depth=best_HP2, random_state=42, n_jobs = -1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best n_estimators: {best_HP1}, Best max_depth: {best_HP2}, Best ACC: {round(best_accuracy,2)}, Time: {iteration_time:.2f} seconds")

end_time2 = time.time()
print(f"Total time: {end_time2 - start_time2:.2f} seconds")

Year 1933 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 77.63 seconds
Year 1934 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 78.17 seconds
Year 1935 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 83.69 seconds
Year 1936 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 88.78 seconds
Year 1937 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 82.11 seconds
Year 1938 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 81.96 seconds
Year 1939 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 82.99 seconds
Year 1940 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 87.81 seconds
Year 1941 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 95.44 seconds
Year 1942 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 97.50 seconds
Year 1943 - Best n_estimators: 10, Best max_depth: 5, Best ACC: 1.0, Time: 98.44 seconds
Year 1944 - Best n_es

## Forming Portfolios, Value-weighted portfolio returns

In [12]:
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,STR,LTURNOVER,IMOM,MOM,LTR,PCTHIGH,IVOL2,IVOL3,MVOL,MVOL2,MVOL3,LMKT2,LMKT3,MMOM,MIMOM,MLTR,LSPREAD,year,ols_default,ridge_reg_roll5,lasso_reg_roll5,DT_reg_roll5,RF_reg_roll5
0,14314,1927-08-31,-0.164557,3902.25,0.0,0.000000,0.000000,0.083333,0.000000,0.0,0.538722,0.000033,0,0.040267,0.000003,0.551246,0.668133,0.709597,0.443203,0.000036,0.000097,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.002245,1927,NaN,NaN,NaN,NaN,NaN
1,12730,1927-08-31,-0.069853,88107.25,0.0,0.416667,0.000000,0.000000,0.000000,0.0,0.538722,0.000005,0,0.043893,0.000019,0.611572,0.736577,0.697120,0.824047,0.000005,0.000004,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.001956,1927,NaN,NaN,NaN,NaN,NaN
2,11594,1927-08-31,0.149390,14137.50,0.0,0.083333,0.000000,0.000000,0.250000,0.0,0.538722,0.000077,1,0.045470,0.000002,0.572206,0.709391,0.708355,0.869882,0.000005,0.000018,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.005677,1927,NaN,NaN,NaN,NaN,NaN
3,75471,1927-08-31,0.216216,1575.00,0.0,0.000000,0.000000,0.000000,0.083333,0.0,0.538722,0.000090,1,0.040866,0.000003,0.570800,0.709576,0.684379,0.860310,0.000038,0.000123,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.003594,1927,NaN,NaN,NaN,NaN,NaN
4,10786,1927-08-31,0.169811,111600.00,0.0,0.166667,0.583333,0.000000,0.083333,0.0,0.538722,0.000013,1,0.045594,0.000056,0.596852,0.722659,0.703022,1.000000,0.000003,0.000006,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.003555,1927,NaN,NaN,NaN,NaN,NaN


In [13]:
prediction_cols
# prediction_cols = ['logit_default','OLS_default','logit_roll6','DT_reg_roll']

['ols_default',
 'ridge_reg_roll5',
 'lasso_reg_roll5',
 'DT_reg_roll5',
 'RF_reg_roll5']

In [14]:
portfolio = df[['date', 'RET', 'ME', 'y'] + prediction_cols].copy()
portfolio['date'] = pd.to_datetime(portfolio['date'])

# drop rows with missing values
portfolio.dropna(inplace=True)

portfolio.head()

,date,RET,ME,y,ols_default,ridge_reg_roll5,lasso_reg_roll5,DT_reg_roll5,RF_reg_roll5
39215,1933-01-31,0.058824,456.750,1,0.589930,0.293012,0.391048,0.898343,0.882098
39216,1933-01-31,0.427451,18036.000,1,0.945905,0.671171,0.587175,1.000000,1.000000
39217,1933-01-31,-0.006993,1975.125,0,0.231157,0.086876,0.272193,0.000000,0.000000
39218,1933-01-31,0.166667,241.500,1,0.862059,0.599736,0.524790,1.000000,1.000000
39219,1933-01-31,-0.058824,3304.000,0,-0.008792,-0.071598,0.153338,0.000000,0.000000


In [15]:
portfolio.tail()

,date,RET,ME,y,ols_default,ridge_reg_roll5,lasso_reg_roll5,DT_reg_roll5,RF_reg_roll5
3076047,2022-07-31,-0.203046,4.174787e+04,0,-0.838064,-0.688308,-0.227265,0.000000,0.000000
3076048,2022-07-31,0.135593,1.110135e+06,1,0.836580,0.891825,0.544300,0.992545,0.991555
3076049,2022-07-31,0.074970,3.728386e+06,1,0.869929,0.949836,0.686789,1.000000,1.000000
3076050,2022-07-31,-0.029348,1.304941e+05,0,0.048938,0.236569,0.273254,0.000000,0.000000
3076051,2022-07-31,0.323765,9.311106e+08,1,0.825924,0.923811,0.521019,0.992545,0.991555


In [16]:
# Initialize an empty DataFrame to store value-weighted returns for each model
vwreturns = pd.DataFrame(portfolio['date'].unique(), columns=['date'])  # Ensures all dates are included

for pred_col in prediction_cols:
    # Calculate deciles for this prediction
    decile_col = f'decile_{pred_col}'
    portfolio[decile_col] = portfolio.groupby(['date'])[pred_col].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))
    
    # Determine position based on deciles
    position_col = f'position_{pred_col}'
    portfolio[position_col] = np.where(portfolio[decile_col] == 9, 1, np.where(portfolio[decile_col] == 0, -1, 0))
    
    # Calculate the value-weighted return for this prediction
    vwret_col = f'vwreturn_{pred_col}'
    vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
    
    # Merge the temporary value-weighted returns with the main vwreturns DataFrame
    vwreturns = vwreturns.merge(vwreturns_temp, on='date', how='left')

# Ensure the 'date' column is the first column and is sorted
vwreturns = vwreturns.sort_values('date').reset_index(drop=True)


/tmp/ipykernel_18/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
/tmp/ipykernel_18/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[positi

In [17]:
vwreturns.head()

,date,vwreturn_ols_default,vwreturn_ridge_reg_roll5,vwreturn_lasso_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_RF_reg_roll5
0,1933-01-31,0.018723,0.019233,0.003639,0.021885,0.021885
1,1933-02-28,0.011374,0.012472,0.010165,0.145270,0.145404
2,1933-03-31,0.019686,0.020244,0.017215,0.027125,0.027125
3,1933-04-30,0.103667,0.050334,-0.098768,-0.000179,-0.000179
4,1933-05-31,0.022140,0.020998,-0.004763,-0.004281,-0.004281


### Compare to market data

In [18]:
#market = pd.read_csv('FF3_clean.csv')
market = pd.read_csv('/kaggle/input/sign-prediction-datasets/FF3_clean.csv')

In [19]:
market.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-31,2.96,-2.56,-2.43,0.22
1,1926-08-31,2.64,-1.17,3.82,0.25
2,1926-09-30,0.36,-1.40,0.13,0.23
3,1926-10-31,-3.24,-0.09,0.70,0.32
4,1926-11-30,2.53,-0.10,-0.51,0.31


In [20]:
# create a new 'Mkt' which is a sum of Mkt-RF and RF
market['Mkt'] = market['Mkt-RF'] + market['RF']

# divide all columns by 100 except 'date'
market.iloc[:, 1:] = market.iloc[:, 1:] / 100

#set the 'date' column to datetime format
market['date'] = pd.to_datetime(market['date'])

# merge the market data (only date and Mkt columns) with the vwreturns DataFrame
vwreturns = vwreturns.merge(market[['date', 'Mkt']], on='date', how='left')

# transform all columns (except 'date') to a log: log(x+1) and save the result as lvwreturns
lvwreturns = vwreturns.copy()
lvwreturns.iloc[:, 1:] = np.log(vwreturns.iloc[:, 1:] + 1)

In [21]:
vwreturns.head()

,date,vwreturn_ols_default,vwreturn_ridge_reg_roll5,vwreturn_lasso_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_RF_reg_roll5,Mkt
0,1933-01-31,0.018723,0.019233,0.003639,0.021885,0.021885,0.0126
1,1933-02-28,0.011374,0.012472,0.010165,0.145270,0.145404,-0.1527
2,1933-03-31,0.019686,0.020244,0.017215,0.027125,0.027125,0.0333
3,1933-04-30,0.103667,0.050334,-0.098768,-0.000179,-0.000179,0.3895
4,1933-05-31,0.022140,0.020998,-0.004763,-0.004281,-0.004281,0.2147


In [22]:
lvwreturns.head()

,date,vwreturn_ols_default,vwreturn_ridge_reg_roll5,vwreturn_lasso_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_RF_reg_roll5,Mkt
0,1933-01-31,0.018550,0.019051,0.003633,0.021649,0.021649,0.012521
1,1933-02-28,0.011309,0.012395,0.010114,0.135640,0.135758,-0.165700
2,1933-03-31,0.019495,0.020041,0.017068,0.026763,0.026763,0.032758
3,1933-04-30,0.098638,0.049108,-0.103992,-0.000179,-0.000179,0.328944
4,1933-05-31,0.021899,0.020781,-0.004774,-0.004290,-0.004290,0.194497


In [23]:
lvwreturns.describe()

,date,vwreturn_ols_default,vwreturn_ridge_reg_roll5,vwreturn_lasso_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_RF_reg_roll5,Mkt
count,1072,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000
mean,1977-11-17 20:54:37.611940288,0.014632,0.014737,0.004372,0.016735,0.019538,0.009039
min,1933-01-31 00:00:00,-0.046196,-0.047609,-0.166080,-0.212917,-0.082288,-0.272203
25%,1955-05-23 06:00:00,0.009861,0.009820,0.002087,0.002923,0.004841,-0.017248
50%,1977-12-15 12:00:00,0.013339,0.013344,0.003648,0.011560,0.013384,0.013262
75%,2000-04-07 12:00:00,0.017580,0.017959,0.006623,0.026238,0.027280,0.038235
max,2022-07-31 00:00:00,0.098638,0.080937,0.107459,0.207777,0.207777,0.328944
std,NaN,0.007952,0.007846,0.011262,0.027774,0.024676,0.048197


In [24]:
# save the lvwreturns and portfolio DataFrame to a parquet file into 'outputs' folder

# for reproducibility and visualization purposes
lvwreturns.to_parquet('market_lvwreturns_reg1.parquet')
portfolio.to_parquet('market_portfolio_reg1.parquet')

# save vwreturns DataFrame to a .dta file into 'outputs' folder
#vwreturns.to_stata('outputs/vwreturns.dta') # for backtasting in R - we need normal returns, not log returns
